In [1]:
#%matplotlib widget

In [2]:
import sys
sys.path.append('/mnt/d/ariel2/code/core/')
sys.path.append('d:/ariel2/code/core/')
sys.path.append('/kaggle/input/my-ariel2-library')
import kaggle_support as kgs
import ariel_model
import ariel_simple2
import ariel_gp
import ariel_load
import ariel_load_FGS
import ariel_load_AIRS
import ariel_pca
import os
import numpy as np
import matplotlib.pyplot as plt
import copy
import importlib
from tqdm import tqdm
import seaborn as sns
import pandas as pd
kgs.debugging_mode = 1
kgs.profiling=False
kgs.n_workers = 6

(3, 32, 282)
(32, 282, 3)
(3, 32, 282)


In [3]:
train_data = kgs.load_all_train_data()
train_data = train_data
# t = []
# for ind in range(len(train_data)):
#     if ind not in [999, 114, 21, 647, 146, 147, 897, 364, 553, 1013, 264, 90, 931, 697, 501, 288, 689, 892, 64, 251, 298, 876, 886, 494, 220, 398, 16, 727, 277, 1067, 1021, 240, 1002, 809, 460, 665, 679, 729, 1049, 989, 383, 531, 305, 1025, 401, 838, 99, 163, 1055, 695, 894, 750, 111, 418, 328, 403, 680, 171, 683, 877, 761, 765, 956, 119, 618, 216, 60, 9, 0, 159, 1091, 112, 422, 885, 229, 140, 568, 787, 11, 686, 563, 53, 1063, 636, 107, 737, 700, 795, 646, 1022, 712, 375, 333, 516, 185, 441, 656, 237, 262, 306, 408, 992, 893, 869, 954, 978, 285, 487, 601, 642, 245, 72, 484, 582, 1065, 933, 129, 591, 476, 393, 430, 784, 373, 722, 965, 263, 366, 379, 807, 479, 570, 349, 760, 782, 184, 1030, 998, 546, 337, 888, 477, 547, 23, 371, 678, 311, 669, 440, 505, 196, 720, 425, 75, 174, 44, 969, 100, 78, 120, 475, 578, 361, 536, 653, 688, 810, 1058, 213, 294, 96, 1031, 758, 708, 733, 362, 135, 734, 847, 485, 619, 17, 709, 161, 823, 826, 677, 152, 794, 1028, 812, 739, 1001, 907, 19, 879, 698, 202, 192, 793, 1024, 467, 803, 870, 66, 178, 416, 801, 816, 658, 592, 249, 439, 663, 160, 1047, 492, 786, 74, 604, 993, 219, 673, 593, 59, 836, 1088, 1075, 325, 588, 208, 151, 961, 913, 873, 321, 740, 951, 1066, 514, 271, 799, 137, 206, 675, 518, 724, 508, 341, 735, 387, 555, 864, 935, 538, 815, 198, 180, 549, 820, 327, 93, 899, 390, 667, 117, 903, 1057, 906, 701, 87, 909, 470, 421, 449, 290, 47, 88, 641, 783, 270, 754, 128, 703, 404, 845, 1004, 385, 314, 842, 1027, 606, 378, 228, 257, 101, 338, 193, 1000, 829, 519, 529, 156, 209, 13, 497, 1046, 438, 230, 624, 704, 970, 496, 164, 772, 70, 692, 384, 736, 824, 1094, 136, 948, 1051, 639, 901, 307, 912, 650, 1054, 696, 279, 169, 199, 22, 1042, 269, 201, 796, 278, 1036, 941, 891, 396, 716, 410, 920, 575, 255, 778, 1081, 925, 562, 963, 372, 367, 212, 567, 1038, 472, 571, 730, 153, 916, 144, 942, 442, 1095, 613, 1092, 713, 92, 1079, 910, 166, 89, 629, 2, 884, 177, 651, 447, 702, 655, 344, 203, 725, 412, 319, 622, 602, 572, 108, 448, 27, 217, 323, 461, 149, 865, 975, 18, 83, 875, 973, 530, 62, 732, 145, 603, 915, 902, 950, 431, 972, 97, 959, 693, 415, 246, 839, 874, 1093, 343, 28, 905, 286, 86, 445, 738, 871, 611, 962, 503, 806, 856, 292, 468, 256, 539, 122, 348, 459, 515, 707, 374, 466, 1015, 493, 825, 881, 1084, 615, 46, 31, 58, 427, 576, 500, 747, 797, 483, 790, 406, 690, 889, 197, 79, 805, 977, 605, 517, 155, 682, 559, 581, 423, 315, 284, 76, 830, 310, 313, 7, 946, 513, 584, 347, 560, 1006, 932, 649, 1052, 424, 138, 26, 247, 749, 640, 955, 533, 580, 499, 1034, 666, 200, 785, 1040, 645, 1041, 545, 317, 345, 30, 543, 952, 259, 8, 464, 296, 148, 369, 187, 995, 242, 880, 426, 322, 757, 1087, 607, 450, 743, 14, 167, 265, 1008, 131, 69, 471, 55, 126, 1062, 91, 481, 831, 486, 623, 726, 985, 35, 282, 173, 346, 186, 1043, 37, 990, 1048, 813, 670, 746, 1017, 363, 252, 504, 612, 32, 718, 569, 326, 768, 866, 1064, 110, 631, 420, 332, 142, 20, 643, 1016, 742, 443, 984, 986, 1069, 828, 551, 510, 316, 234, 921, 261, 744, 764, 851, 154, 859, 561, 1020, 583, 253, 940, 821, 684, 254, 832, 699, 846, 918, 50, 949, 565, 172, 81, 473, 42, 506, 1053, 896, 634, 488, 706, 774, 113, 710, 352, 376, 526, 1039, 857, 1086, 304, 862, 1050, 1083, 218, 957, 48, 127, 102, 280, 51, 895, 359, 858, 238, 1090, 324, 105, 930, 776, 210, 1071, 632, 741, 444, 405, 944, 339, 791, 964, 971, 745, 189, 370, 293, 224, 1033, 437, 960, 241, 759, 236, 115, 1023, 861, 301, 509, 586, 103, 190, 715, 239, 451, 205, 77, 150, 434, 104, 409, 756, 157, 927, 853, 924, 1056, 121, 223, 250, 318, 56, 542, 1, 751, 600, 608, 926, 994, 878, 1011, 226, 585, 635, 182, 436, 541, 522, 266, 482, 227, 132, 819, 911, 779, 798, 388, 134, 1098, 287, 162, 15, 1035, 52, 65, 395, 94, 773, 843, 1096, 934, 685, 480, 428, 590, 235, 867, 130, 728, 175, 433, 243, 106, 868, 399, 1029, 411, 456, 633, 616, 368, 281, 674, 900, 637, 498, 1072, 827, 57, 188, 143, 664, 354, 991, 158, 817, 534, 351, 392, 495, 25, 630, 380, 24, 676, 802, 1007, 231, 29, 947, 1018, 1073, 248, 232, 939, 863, 850, 691, 587, 356, 1085, 489, 33, 4, 407, 195, 943, 1070, 54, 579, 609, 835, 917, 353, 457, 520, 537, 748, 833, 837, 276, 469, 525, 980, 1019, 40, 1089, 465, 771, 272, 331, 80, 966, 6, 299, 852, 511, 808, 233, 68, 781, 67, 919, 260, 413, 769, 302, 654, 139, 183, 303, 557, 360, 532, 308, 41, 1009, 890, 453, 694, 458, 723, 12, 979, 1082, 312, 714, 168, 540, 49, 620, 860, 381, 717, 267, 594, 804, 1044, 377, 770, 141, 854, 336, 342, 898, 400, 883, 814, 84, 626, 625, 659, 752, 507, 61, 904, 391, 268, 211, 512, 221, 283, 687, 124, 3, 116, 638, 958, 614, 214, 1074, 179, 988, 953, 454, 844, 822, 98, 1037, 1060, 660, 358, 222, 181, 573, 258, 340, 382, 1014, 767, 1026, 550, 225, 937, 45, 291, 355, 191, 194, 922, 762, 300, 320, 766, 274, 474, 389, 661, 657, 295, 1068, 564, 1059, 10, 502, 711, 908, 297, 1078, 841, 1061, 165, 628, 544, 527, 789, 95, 478, 82, 71, 719, 753, 244, 981, 419, 983, 402, 929, 800, 763, 455, 668, 968, 974, 662, 914, 528, 34, 840, 617, 463, 599, 435, 535, 207, 36, 491, 63, 597, 627, 432, 446, 621, 462, 610, 644, 574, 882, 589, 275, 133, 386, 1005, 872, 125, 566, 5, 523, 524, 123, 452, 834, 1076, 1032, 777, 938, 848, 681, 43, 554, 204, 928, 976, 982, 577, 775, 987, 996, 923, 671, 170, 731, 855, 397, 109, 755, 1080, 705, 38]:
#         t.append(train_data[ind])
#         #print(ind)
# train_data = t[:6]
# train_data_multi = []
# train_data_single = []
# for d in train_data:
#     if len(d.transits)==1:
#         train_data_single.append(d)
#     else:
#         train_data_multi.append(d)
# len(train_data_multi), len(train_data_single)

# train_data = train_data_multi

In [4]:
importlib.reload(ariel_model)
import ariel_pca
importlib.reload(ariel_pca)
todo = dict()

model = ariel_model.baseline_model()
# loaders = model.model.model.starter_model.model.loaders
# loaders[1].apply_full_sensor_corrections.pca_options.n_components = 2
# loaders[1].apply_full_sensor_corrections.remove_background_based_on_rows = True
# loaders[1].apply_full_sensor_corrections.inpainting_2d = True
# loaders[1].apply_full_sensor_corrections.restore_invalids = True
# model.model.model.model_options.n_iter = 8
# loaders[1].apply_wavelength_binning = ariel_load_AIRS.ApplyWavelengthBinningAIRS3()
# loaders = model.model.model.starter_model.model.loaders
# print(loaders[1].apply_wavelength_binning.n_iter)
# #loaders[1].apply_wavelength_binning.n_iter=3 
# print(loaders[1].apply_wavelength_binning.n_iter)
todo['New new new new baseline'] = model

# model = ariel_model.baseline_model()
# loaders = model.model.model.starter_model.model.loaders
# loaders[1].apply_full_sensor_corrections.pca_options.n_components = 2
# loaders[1].apply_full_sensor_corrections.remove_background_based_on_rows = True
# loaders[1].apply_full_sensor_corrections.inpainting_2d = True
# loaders[1].apply_full_sensor_corrections.restore_invalids = True
# loaders[1].apply_wavelength_binning = ariel_load_AIRS.ApplyWavelengthBinningAIRS3()
# #todo['New AIRS'] = model

# #todo['New AIRS, calibrated on all'] = copy.deepcopy(model)

# # for ii in [3]:
# #     model = copy.deepcopy(model)
# #     loaders = model.model.model.starter_model.model.loaders
# #     loaders[1].apply_wavelength_binning.n_iter=ii    
# #     todo[f'New AIRS, calibrated on all, {ii} load iters'] = model
    
# model = copy.deepcopy(model)
# model.model.model.model_options.n_iter = 8
# loaders = model.model.model.starter_model.model.loaders
# loaders[1].apply_wavelength_binning.n_iter=3 
# todo['New AIRS, calibrated on all, 3 load iters, 8 model iters X'] = copy.deepcopy(model)

# # model = copy.deepcopy(model)
# # model.model.model.model_options.n_iter = 12
# # loaders = model.model.model.starter_model.model.loaders
# # loaders[1].apply_wavelength_binning.n_iter=ii 
# # todo['New AIRS, calibrated on all, 3 load iters, 12 model iters'] = copy.deepcopy(model)


# # model = ariel_model.baseline_model()
# # def modify_func(model, cc):
# #     model.m['signal'].m['main'].m['transit'].cov_override = cc[0]
# #     model.m['signal'].m['main'].m['transit'].mu_override = cc[1]
# #     model.m['signal'].m['main'].m['transit'].TS_slopes = cc[2]
# #     # cur_params = model.m['signal'].m['main'].m['transit'].get_parameters()
# #     # cur_params[-model.m['signal'].m['main'].m['transit'].number_of_extra_parameters:,0] = cc[1]
# #     # model.m['signal'].m['main'].m['transit'].set_parameters(cur_params)
# #     return model
# # model.model.model.model_options.modify_func = modify_func
# # model.model.model.model_options.modify_func_input = kgs.dill_load(kgs.temp_dir + f'transit_model_tuning_Ts_14.pickle')
# # todo['Ts slopes'] = model

In [5]:

kgs.sanity_checks_active = False
print('Sanity checks off!')
def fname(k):
    return kgs.temp_dir + 'evaluate_model_'+k+'__'+str(len(train_data))+'.pickle'
for k,v in todo.items():
    if not os.path.isfile(fname(k)):
        print(k)
        model = copy.deepcopy(v)
        model.model.model.run_in_parallel = True
       # model.model.run_in_parallel = False
#         print('!!! also loader')
        #if k=='Run on new simple reg kepler fudge_var adjust_u':
            
        #model_ref = kgs.dill_load(fname('All + multifudge'))[2]
        #model._cached_planet_id = model_ref._cached_planet_id
        #model._cached_result = model_ref._cached_result
            # model.bias_a = model_ref.bias_a
            # model.bias_b = model_ref.bias_b
            # model.sigma_fudge_FGS = model_ref.sigma_fudge[0]
            # model.sigma_fudge_AIRS_mean = model_ref.sigma_fudge[1]
            # model.sigma_fudge_AIRS_var = model_ref.sigma_fudge[1]
        
        # Precache data
        #print('precache off')
        #for d in tqdm(train_data):
        #    copy.deepcopy(d).load_to_step(5,model.model.model.starter_model.model.loaders)
        
        
        
        #model.state=1
        model.train(train_data)
        #kgs.list_attrs(model)
        inferred_data = model.infer(train_data)
        print(k)
        print(kgs.score_metric(inferred_data, train_data))
        #print('not saving')
        #kgs.list_attrs(model)
        #if k=='Baseline':
        kgs.dill_save(fname(k), (inferred_data,train_data,model,kgs.git_commit_id))
        #print(inferred_data[0].spectrum[0], inferred_data[0].spectrum_cov[0,0])
        #print(inferred_data[-1].spectrum[0], inferred_data[-1].spectrum_cov[0,0])
    else:
        print(k)
        (inferred_data,train_data_here,model,_) = kgs.dill_load(fname(k))
        #print(model.model.model_options.FGS_order)
        print(kgs.score_metric(inferred_data, train_data_here))

Sanity checks off!
New new new new baseline


Processing in parallel: 100%|█████████████████████████████████████████████████████████| 100/100 [03:14<00:00,  1.95s/it]


SpawnPoolWorker-7 7
CUDA_VISIBLE_DEVICES= 0
(3, 32, 282)
(32, 282, 3)
(3, 32, 282)
Opt time 200.74184012413025
New new new new baseline
Score:           0.6800
RMS error FGS:   83.57270 ppm
mRMS error FGS:  45.74885 ppm
RMS error AIRS:  56.66503 ppm
mRMS error AIRS: 41.12263 ppm
RMS error AIRSv: 40.14070 ppm
mRMS error AIRSv:27.77452 ppm
(0.6800058978828101, np.float64(8.357269565876983e-05), np.float64(5.6665033757025834e-05))


In [6]:
%matplotlib widget
ignore_bad_planets = False
def filter_bad_data(data):
    data_out = []
    for d in data:
        if not d.planet_id in [1349926825, 3786449677, 2554492145, 1267010874, 94572221, 2270815333, 3649218579, 576917580, 2154305089, 2207815333, 2740910036]:
            data_out.append(d)
    return data_out
        
def basics():
    plt.grid(True)
    plt.box(True)
def plot_two_sensors(func, xl, yl, mod):
    _,ax = plt.subplots(1,2,figsize=(12,6))
    plt.sca(ax[0]);basics()
    plt.title('FGS')
    plt.xlabel(xl)
    plt.ylabel(yl)
    sc=func(slice(0,1), 'blue', 'FGS')
    mod()
    kgs.add_cursor(sc)
    
    plt.sca(ax[1]);basics()
    plt.title('AIRS')
    plt.xlabel(xl)
    sc=func(slice(1,283), 'red', 'AIRS')
    mod()
    kgs.add_cursor(sc)
    
    plt.pause(0.001)

def report_extreme_ratios(x,y,name):
    return    
    print(name)
    inds = np.argsort(np.abs(y/x))[::-1]
    for ii in inds[:2]:
        print(f'{inferred_data[ii].planet_id}, ratio:{np.abs(y[ii]/x[ii]):.3f}, x:{x[ii]:.3e}, y:{y[ii]:.3e}, RMS:{kgs.rms(error_matrix[ii,:]):.3e}')
    
    
fname_list = [fname(k) for k in todo.keys()]
#fname_list = ['evaluate_model_Baseline__989.pickle', 'evaluate_model_Baseline__110.pickle', 'evaluate_model_Multi-transit__110.pickle']
for f in fname_list:
    (inferred_data,train_data_here,model,_) = kgs.dill_load(f)
    if ignore_bad_planets:
        inferred_data = filter_bad_data(inferred_data)
        train_data_here = filter_bad_data(train_data_here)
    else:
        train_data_here = train_data_here
    #model.bias_a[0]+=0.006
    #model.bias_a[1]-=0.0002
    #inferred_data = model.infer(train_data)
    
    
    print(f)
    print(model.bias_a)
    print(model.bias_b)
    print(model.sigma_offset)
    print(model.sigma_fudge_FGS, model.sigma_fudge_AIRS_mean, model.sigma_fudge_AIRS_var)
    print('mul', model.sigma_fudge_multi)
    
    print(kgs.score_metric(inferred_data, train_data_here,print_results=True))
    print('')
    
    continue
    
    
    
    
    #try:
    #    print(model.sigma_offset)
    #except: pass
    #print(model.sigma_fudge)    
    #kgs.list_attrs(model)
    #kgs.score_metric(inferred_data, train_data_here)
    print('')
    #print('3')
    #kgs.score_metric(inferred_data[:3], train_data_here[:3]) 
    #print('')
    # print('No fudge')
    # #model.sigma_fudge = [1,1]
    # #try: model.sigma_offset = [0,0]
    # #except: pass
    # #model.bias_a = [1,1]
    # #model.bias_b = [0,0]
    # # print('!!!')
    # model.sigma_offset = [0,0]
    # model.sigma_fudge_FGS = 1.
    # model.sigma_fudge_AIRS_mean = 1.
    # model.sigma_fudge_AIRS_var = 1.
    # inferred_data = model.infer(train_data_here)
    #if ignore_bad_planets:
    #    inferred_data = filter_bad_data(inferred_data)
    #kgs.score_metric(inferred_data, train_data_here)     
    error_matrix = np.array([d.spectrum - t.spectrum for d,t in zip(inferred_data,train_data_here)])
    true_matrix = np.array([t.spectrum for d,t in zip(inferred_data,train_data_here)])
    
    T1 = []
    T4 = []
    for d in inferred_data:
        trans = d.diagnostics['transit_params_gp'][1]
        trans.Rp = ariel_gp.msqrtabs(-np.mean(d.spectrum))
        times = np.linspace(-3,10,10000)
        light_curve = trans.light_curve(times)
        assert(light_curve[0]==1)
        assert(light_curve[1]==1)
        T1.append(times[np.argwhere(light_curve!=1)[0][0]])
        T4.append(times[np.argwhere(light_curve!=1)[-1][0]])
    T1 = np.array(T1)
    T4 = np.array(T4)
        
#     def plot_mean_error_vs_sigma(slic,col, name):
#         x = T1
#         y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
#         #y = np.mean(error_matrix[:,slic],1)
#         sc=plt.scatter(x,y,color = col)
#         #print(y[934])
#         report_extreme_ratios(x,y,name)    
#         return sc
#     plot_two_sensors(plot_mean_error_vs_sigma, 'T1 [h]', 'Prediction error / sigma (mean)', lambda:0)
    
#     def plot_mean_error_vs_sigma(slic,col, name):
#         x = T4
#         y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
#         #y = np.mean(error_matrix[:,slic],1)
#         sc=plt.scatter(x,y,color = col)
#         #print(y[934])
#         report_extreme_ratios(x,y,name)    
#         return sc
#     plot_two_sensors(plot_mean_error_vs_sigma, 'T4 [h]', 'Prediction error / sigma (mean)', lambda:0)
    
#     def plot_mean_error_vs_sigma(slic,col, name):
#         x = np.argsort(np.argsort(T4-T1))
#         y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
#         #y = np.mean(error_matrix[:,slic],1)
#         sc=plt.scatter(x,y,color = col)
#         #print(y[934])
#         report_extreme_ratios(x,y,name)    
#         return sc
#     plot_two_sensors(plot_mean_error_vs_sigma, 'T4-T1 rank', 'Prediction error / sigma (mean)', lambda:0)
    
#     def plot_mean_error_vs_sigma(slic,col, name):
#         x = T4-T1
#         y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
#         #y = np.mean(error_matrix[:,slic],1)
#         sc=plt.scatter(x,y,color = col)
#         #print(y[934])
#         report_extreme_ratios(x,y,name)    
#         return sc
#     plot_two_sensors(plot_mean_error_vs_sigma, 'T4-T1 [h]', 'Prediction error / sigma (mean)', lambda:0)
    
#    continue
    
#     def plot_mean_error_vs_sigma(slic,col, name):
#         x = [d.transit_params.Rs for d in train_data_here]
#         y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
#         #y = np.mean(error_matrix[:,slic],1)
#         sc=plt.scatter(x,y,color = col)
#         #print(y[934])
#         report_extreme_ratios(x,y,name)    
#         return sc
#     plot_two_sensors(plot_mean_error_vs_sigma, 'Rs', 'Prediction error / sigma (mean)', lambda:0)
#     def plot_mean_error_vs_sigma(slic,col, name):
#         x = [d.transit_params.Ts for d in train_data_here]
#         y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
#         #y = np.mean(error_matrix[:,slic],1)
#         sc=plt.scatter(x,y,color = col)
#         #print(y[934])
#         report_extreme_ratios(x,y,name)    
#         return sc
#     plot_two_sensors(plot_mean_error_vs_sigma, 'Ts', 'Prediction error / sigma (mean)', lambda:0)
#     def plot_mean_error_vs_sigma(slic,col, name):
#         x = [d.transit_params.Mp for d in train_data_here]
#         y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
#         #y = np.mean(error_matrix[:,slic],1)
#         sc=plt.scatter(x,y,color = col)
#         #print(y[934])
#         report_extreme_ratios(x,y,name)    
#         return sc
#     plot_two_sensors(plot_mean_error_vs_sigma, 'Mp', 'Prediction error / sigma (mean)', lambda:0)
#     def plot_mean_error_vs_sigma(slic,col, name):
#         x = [d.transit_params.Ms for d in train_data_here]
#         y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
#         #y = np.mean(error_matrix[:,slic],1)
#         sc=plt.scatter(x,y,color = col)
#         #print(y[934])
#         report_extreme_ratios(x,y,name)    
#         return sc
#     plot_two_sensors(plot_mean_error_vs_sigma, 'Ms', 'Prediction error / sigma (mean)', lambda:0)
    
 #   continue
    
#     def plot_mean_error_vs_sigma(slic,col, name):
#         x = [len(d.transits) for d in inferred_data]
#         y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
#         #y = np.mean(error_matrix[:,slic],1)
#         sc=plt.scatter(x,y,color = col)
#         #print(y[934])
#         report_extreme_ratios(x,y,name)    
#         return sc
#     plot_two_sensors(plot_mean_error_vs_sigma, 'Number of transits', 'Prediction error / sigma (mean)', lambda:0)
    
    #conintue
    
    def plot_prediction_vs_true(slic, col, name):
        x = np.mean(true_matrix[:,slic],1)
        y = np.mean(error_matrix[:,slic],1)
        sc=plt.scatter(x,y,color = col)
        report_extreme_ratios(x,y,name)
        return sc
        
    plot_two_sensors(plot_prediction_vs_true, 'True value (mean)', 'Prediction error (mean)', lambda:0)
    
    def plot_mean_error_vs_sigma(slic,col, name):
        x = [np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data]
        y = np.mean(error_matrix[:,slic],1)
        sc=plt.scatter(x,y,color = col)
        report_extreme_ratios(x,y,name)
        return(sc)
    def mod():
        plt.axline((0,0), slope=3, color='black')
        plt.axline((0,0), slope=-3, color = 'black')        
    plot_two_sensors(plot_mean_error_vs_sigma, 'Sigma prediction (mean)', 'Prediction error (mean)', mod)
    
    def plot_mean_error_vs_sigma(slic,col, name):
        x = np.mean(true_matrix[:,slic],1)
        y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
        plt.scatter(x,y,color = col)
        report_extreme_ratios(x,y,name)    
    plot_two_sensors(plot_mean_error_vs_sigma, 'True value (mean)', 'Prediction error / sigma (mean)', lambda:0)
    
    def plot_mean_error_vs_sigma(slic,col, name):
        x = np.std(true_matrix[:,1:],1)
        y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
        #y = np.mean(error_matrix[:,slic],1)
        sc=plt.scatter(x,y,color = col)
        report_extreme_ratios(x,y,name)    
        return sc
    plot_two_sensors(plot_mean_error_vs_sigma, 'AIRS variation', 'Prediction error / sigma (mean)', lambda:0)
    

    
#     def plot_mean_error_vs_sigma(slic,col, name):
#         x = [d.diagnostics['simple_residual_diff_AIRS'] for d in inferred_data]
#         y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
#         #y = np.mean(error_matrix[:,slic],1)
#         sc=plt.scatter(x,y,color = col)
#         report_extreme_ratios(x,y,name)    
#         return sc
#     plot_two_sensors(plot_mean_error_vs_sigma, 'AIRS residual ratio', 'Prediction error / sigma (mean)', lambda:0)
    
#     def plot_mean_error_vs_sigma(slic,col, name):
#         x = [d.diagnostics['simple_residual_diff_FGS'] for d in inferred_data]
#         y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
#         #y = np.mean(error_matrix[:,slic],1)
#         sc=plt.scatter(x,y,color = col)
#         #print(y[934])
#         report_extreme_ratios(x,y,name)    
#         return sc
#     plot_two_sensors(plot_mean_error_vs_sigma, 'FGS residual ratio', 'Prediction error / sigma (mean)', lambda:0)
    
    plt.figure(); basics()
    plt.title('AIRS variation per planet')
    plt.xlabel('Sigma prediction')
    plt.ylabel('STD error')
    n=282
    x=np.sqrt([(np.trace(d.spectrum_cov[1:,1:]) - np.sum(d.spectrum_cov[1:,1:]) / n) / n for d in inferred_data])
    y=np.sqrt(np.var(error_matrix[:,1:],1))
    plt.scatter(x, y, color='red')
    plt.axline((0,0), slope=1, color='black')
    report_extreme_ratios(x,y,'AIRS variation')    
    plt.pause(0.001)
    
    continue
    
#     def plot_mean_error_vs_sigma(slic,col, name):
#         x = np.mean(true_matrix[:,slic],1)
#         y = np.sqrt(np.var(error_matrix[:,1:],1)) / np.sqrt([(np.trace(d.spectrum_cov[1:,1:]) - np.sum(d.spectrum_cov[1:,1:]) / n) / n for d in inferred_data])
#         plt.scatter(x,y,color = col)
#         report_extreme_ratios(x,y,name)    
#     plot_two_sensors(plot_mean_error_vs_sigma, 'True value (mean)', 'Prediction error / sigma (AIRS variation)', lambda:0)
    
#     def plot_mean_error_vs_sigma(slic,col, name):
#         x = np.std(true_matrix[:,1:],1)
#         y = np.sqrt(np.var(error_matrix[:,1:],1)) / np.sqrt([(np.trace(d.spectrum_cov[1:,1:]) - np.sum(d.spectrum_cov[1:,1:]) / n) / n for d in inferred_data])
#         sc=plt.scatter(x,y,color = col)
#         report_extreme_ratios(x,y,name)    
#         return sc
#     plot_two_sensors(plot_mean_error_vs_sigma, 'AIRS variation', 'Prediction error / sigma (AIRS variation)', lambda:0)
    
    
    
#     for ii in range(len(inferred_data[0].diagnostics['transit_params_gp_par'])):
#         def plot_mean_error_vs_sigma(slic,col, name):
#             x = [d.diagnostics['transit_params_gp_par'][ii] for d in inferred_data]
#             y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
#             #y = np.mean(error_matrix[:,slic],1)
#             sc=plt.scatter(x,y,color = col)
#             #print(y[934])
#             report_extreme_ratios(x,y,name)    
#             return sc
#         plot_two_sensors(plot_mean_error_vs_sigma, f'Transit param {ii}', 'Prediction error / sigma (mean)', lambda:0)
    
    
    
    # for ii in range(7):
    #     if ii==4:
    #         continue
    #     def plot_mean_error_vs_sigma(slic,col, name):
    #         x = [d.diagnostics['transit_params_gp'][0].to_x()[ii] for d in inferred_data]
    #         y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
    #         #y = np.mean(error_matrix[:,slic],1)
    #         sc=plt.scatter(x,y,color = col)
    #         #print(y[934])
    #         report_extreme_ratios(x,y,name)    
    #         return sc
    #     plot_two_sensors(plot_mean_error_vs_sigma, f'Transit param FGS {ii}', 'Prediction error / sigma (mean)', lambda:0)
        
    # for ii in np.arange(5,7):
    #     def plot_mean_error_vs_sigma(slic,col, name):
    #         x = [d.diagnostics['transit_params_gp'][1].to_x()[ii] for d in inferred_data]
    #         y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
    #         #y = np.mean(error_matrix[:,slic],1)
    #         sc=plt.scatter(x,y,color = col)
    #         #print(y[934])
    #         report_extreme_ratios(x,y,name)    
    #         return sc
    #     plot_two_sensors(plot_mean_error_vs_sigma, f'Transit param AIRS {ii}', 'Prediction error / sigma (mean)', lambda:0)
    
    def plot_mean_error_vs_sigma(slic,col, name):
        x = [d.diagnostics['transit_params_gp'][1].t0-d.diagnostics['transit_params_gp'][0].t0 for d in inferred_data]
        y = np.mean(error_matrix[:,slic],1) / np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in inferred_data])
        #y = np.mean(error_matrix[:,slic],1)
        sc=plt.scatter(x,y,color = col)
        #print(y[934])
        report_extreme_ratios(x,y,name)    
        return sc
    plot_two_sensors(plot_mean_error_vs_sigma, f't0 diff', 'Prediction error / sigma (mean)', lambda:0)
    
    mat = np.stack([d.diagnostics['transit_params_gp_par'] for d in inferred_data])[:,:,0]
    mat.shape
    
    plt.figure()
    plt.imshow(np.corrcoef(mat.T))
    plt.colorbar()
    sns.pairplot(pd.DataFrame(mat), diag_kind="hist", corner=True, height=1.5)
    
    for ii in range(inferred_data[0].diagnostics['transit_params_gp_par_cov'].shape[0]):
        plt.figure()
        plt.plot( np.sqrt([d.diagnostics['transit_params_gp_par_cov'][ii,ii] for d in inferred_data]))
        plt.title(ii-1)
        #plt.plot( np.sqrt([d.diagnostics['transit_params_gp_par_cov'][6,6] for d in inferred_data]))
    for ii in range(2):
        for jj in range(7):
            if jj==4:
                continue
            x = [d.diagnostics['transit_params'][ii].to_x()[jj] for d in inferred_data]
            y = [d.diagnostics['transit_params_gp'][ii].to_x()[jj] for d in inferred_data]
            plt.figure()
            plt.scatter(x,y)
            plt.title(str(ii) + ' ' + str(jj))
            plt.xlabel('Simple')
            plt.ylabel('GP')
            plt.axline((x[0],x[0]), slope=1, color='black')

/mnt/d//ariel2/temp/evaluate_model_New new new new baseline__100.pickle
[np.float64(0.9996403808188005), np.float64(0.9992047870405517)]
[np.float64(6.275457109734808e-05), np.float64(0.00015406485490663174)]
[np.float64(5.8453542376251304e-05), np.float64(2.514747324988365e-05)]
1.1554875444814257 0.9505012016462104 1.3919590967922024
mul 1.0
Score:           0.6800
RMS error FGS:   83.57270 ppm
mRMS error FGS:  45.74885 ppm
RMS error AIRS:  56.66503 ppm
mRMS error AIRS: 41.12263 ppm
RMS error AIRSv: 40.14070 ppm
mRMS error AIRSv:27.77452 ppm
(0.6800058978828101, np.float64(8.357269565876983e-05), np.float64(5.6665033757025834e-05))



In [7]:
compare1= list(todo.keys())[0]
data1 = kgs.dill_load(fname(compare1))[0]
if ignore_bad_planets:
    data1 = filter_bad_data(data1)
for k,v in todo.items():    
    compare2= k
    if compare2==compare1:
        continue    
    data2 = kgs.dill_load(fname(compare2))[0]
    if ignore_bad_planets:
        data2 = filter_bad_data(data2)
        train_data_here = filter_bad_data(train_data)
    else:
        train_data_here = train_data           
    error1 = np.array([d.spectrum - t.spectrum for d,t in zip(data1,train_data_here)])
    error2 = np.array([d.spectrum - t.spectrum for d,t in zip(data2,train_data_here)])
    def plot_prediction_vs_true(slic, col, name):
        x = np.mean(error1[:,slic],1)
        y = np.mean(error2[:,slic],1)
        sc=plt.scatter(x,y,color = col)
        #kgs.add_cursor(sc)
        return sc

    plot_two_sensors(plot_prediction_vs_true, compare1+' (mean)', compare2+' (mean)', lambda:(plt.axline((0,0),slope=1,color='black'),plt.axline((0,0),slope=-1,color='black')))
    
    def plot_prediction_vs_true(slic, col, name):
        x = np.mean(error1[:,slic],1)/np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in data1])
        y = np.mean(error2[:,slic],1)/np.array([np.sqrt(np.sum(d.spectrum_cov[slic,slic])/(slic.stop-slic.start)**2) for d in data2])
        #print(x[934],y[934])
        sc=plt.scatter(x,y,color = col)
        return sc
        #kgs.add_cursor(sc)
    plot_two_sensors(plot_prediction_vs_true, compare1+' pe/s (mean)', compare2+' pe/s (mean)', lambda:(plt.axline((0,0),slope=1,color='black'),plt.axline((0,0),slope=-1,color='black')))
    
#     continue
    
#     for ii in range(2):
#         for jj in range(7):
#             if jj==4:
#                 continue
#             # x = [d.diagnostics['transit_params'][ii].to_x()[jj] for d in data1]
#             # y = [d.diagnostics['transit_params'][ii].to_x()[jj] for d in data2]
#             # plt.figure()
#             # sc=plt.scatter(x,y)
#             # plt.axline((x[0],x[0]), slope=1, color='black')
#             # plt.xlabel(compare1)
#             # plt.ylabel(compare2)
#             # plt.title(str(ii) + ' ' + str(jj))
#             # plt.grid(True)
#             # kgs.add_cursor(sc)
#             x = [d.diagnostics['transit_params_gp'][ii].to_x()[jj] for d in data1]
#             y = [d.diagnostics['transit_params_gp'][ii].to_x()[jj] for d in data2]
#             plt.figure()
#             sc=plt.scatter(x,y)
#             plt.axline((x[0],x[0]), slope=1, color='black')
#             plt.xlabel(compare1)
#             plt.ylabel(compare2)
#             plt.title(str(ii) + ' ' + str(jj) + ' gp')
#             plt.grid(True)
#             kgs.add_cursor(sc)
            
#     x = [d.diagnostics['simple_residual_diff_FGS'] for d in data1]
#     y = [d.diagnostics['simple_residual_diff_FGS'] for d in data2]
#     plt.figure()
#     plt.scatter(x,y)
#     plt.axline((x[0],x[0]), slope=1, color='black')
#     plt.xlabel(compare1)
#     plt.ylabel(compare2)
#     plt.title('FGS residual')
#     plt.grid(True)
    
#     x = [d.diagnostics['simple_residual_diff_AIRS'] for d in data1]
#     y = [d.diagnostics['simple_residual_diff_AIRS'] for d in data2]
#     plt.figure()
#     plt.scatter(x,y)
#     plt.axline((x[0],x[0]), slope=1, color='black')
#     plt.xlabel(compare1)
#     plt.ylabel(compare2)
#     plt.title('AIRS residual')
#     plt.grid(True)
            

    plt.figure(); basics()
    plt.title('AIRS variation per planet')
    plt.xlabel(compare1)
    plt.ylabel(compare2)
    x=np.sqrt(np.var(error1[:,1:],1))
    y=np.sqrt(np.var(error2[:,1:],1))
    plt.scatter(x, y, color='red')
    plt.axline((0,0), slope=1, color='black')
    plt.pause(0.001)

In [8]:
# import subprocess

# subprocess.run(
#     ["/mnt/c/Windows/System32/shutdown.exe", "/h"],  # add "/f" to force-close apps
#     check=True
# )